# TKO_7092 Evaluation of Machine Learning Methods 2025

---

Student name: Muhammad Junaid Raza

Student number: 2409917

Student email: mjraza@utu.fi

---

## Exercise 3

Complete the tasks given to you in the letter below. In your submission, explain clearly, precisely, and comprehensively why the cross-validation described in the letter failed, what is the correct way to perform cross-validation in the given scenario, and why the correct cross-validation method will give a reliable estimate of the generalisation performance. Then implement the correct cross-validation for the scenario and report its results.

Remember to follow all the general exercise guidelines that are stated in Moodle. Full points (2p) will be given for a submission that demonstrates a deep understanding of cross-validation on pair-input data and implements the requested cross-validation correctly (incl. reporting the results). Partial points (1p) will be given if there are small error(s) but the overall approach is correct. No points will be given if there are significant error(s).

The deadline of this exercise is **Wednesday 19 February 2025 at 11:59 PM**. Please contact Juho Heimonen (juaheim@utu.fi) if you have any questions about this exercise.

---


Dear Data Scientist,

I have a long-term research project regarding a specific set of proteins. I am attempting to discover small organic compounds that can bind strongly to these proteins and thus act as drugs. I have already made laboratory experiments to measure the affinities between some proteins and drug molecules.

My colleague is working on another set of proteins, and the objectives of his project are similar to mine. He has recently discovered thousands of new potential drug molecules. He asked me if I could find the pairs that have the strongest affinities among his proteins and drug molecules. Obviously I do not have the resources to measure all the possible pairs in my laboratory, so I need to prioritise. I decided to do this with the help of machine learning, but I have encountered a problem.

Here is what I have done so far: First I trained a K-nearest neighbours regressor with the parameter value K=10 using all the 400 measurements I had already made in the laboratory with my proteins and drug molecules. They comprise of 77 target proteins and 59 drug molecules. Then I performed a leave-one-out cross-validation with this same data to estimate the generalisation performance of the model. I used C-index and got a stellar score above 90%. Finally I used the model to predict the affinities of my colleague's proteins and drug molecules. The problem is: when I selected the highest predicted affinities and tried to verify them in the lab, I found that many of them are much lower in reality. My model clearly does not work despite the high cross-validation score.

Please explain why my estimation failed and how leave-one-out cross-validation should be performed to get a reliable estimate. Also, implement the correct leave-one-out cross-validation and report its results. I need to know whether it would be a waste of my resources if I were to use my model any further.

The data I used to create my model is available in the files `input.data`, `output.data` and `pairs.data` for you to use. The first file contains the features of the pairs, whereas the second contains their affinities. The third file contains the identifiers of the drug and target molecules of which the pairs are composed. The files are paired, i.e. the i<sup>*th*</sup> row in each file is about the same pair.

Looking forward to hearing from you soon.

Yours sincerely, \
Bio Scientist

---

#### Answer the questions about cross-validation on pair-input data

In [ ]:
# Why did the estimation described in the letter fail?
# How should leave-one-out cross-validation be performed in the given scenario and why?
# Remember to provide comprehensive and precise arguments.

<font color = green>1. The estimation failed because the leave-one-out cross-validation (LOO-CV) was performed in a way that did not properly account for the dependencies between the data points. In the original approach, each drug-protein pair was treated as an independent sample, meaning that when one pair was left out for testing, the model was trained on all remaining pairs—even those that included the same protein or drug. This led to data leakage, where the model effectively "saw" part of the test data during training, making the validation performance overly optimistic. The high C-index obtained in this flawed LOO-CV did not reflect the actual generalization ability of the model to completely new drug-protein pairs, leading to poor real-world predictive performance. <br>
2. In this scenario, a Leave-One-Protein-Out Cross-Validation (LOPO-CV) should be used instead of standard LOO-CV. This means that when evaluating a test set, the model should be trained only on data from entirely different proteins, ensuring that it does not learn from interactions involving the same proteins seen during training. The same reasoning applies to drugs: a Leave-One-Drug-Out Cross-Validation (LODO-CV) would assess the model’s ability to generalize to unseen drug molecules. A Leave-One-Group-Out (LOGO-CV) approach, where both drugs and proteins are left out in turn, would provide the most robust estimate of generalization. These approaches prevent data leakage and better simulate real-world scenarios where the goal is to predict binding affinities for completely new proteins and drugs.

#### Import libraries

In [54]:
# Import the libraries you need.
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import LeaveOneGroupOut

#### Write utility functions

In [56]:
# Write the utility functions you need in your analysis.
# Define function to compute C-index
def compute_c_index(true_values, predicted_values):
    n = 0
    h = 0
    for i in range(len(true_values)):
        for j in range(i + 1, len(true_values)):
            if true_values[i] != true_values[j]:  # Only consider pairs with different true values
                n += 1
                if (predicted_values[i] > predicted_values[j]) == (true_values[i] > true_values[j]):
                    h += 1
    return h / n if n > 0 else 0.5

#### Load datasets

In [77]:
# Read the data files (input.data, output.data, pairs.data).

X = np.loadtxt("input.data")  # Feature matrix
y = np.loadtxt("output.data")  # Target values

# Load drug-protein pairs (pairs) using pandas (since it contains strings)
pairs_df = pd.read_csv("pairs.data", delimiter=" ", header=None, dtype=str)

# Extract drug and protein IDs
drugs = pairs_df.iloc[:, 0].values  # First column: Drug IDs
proteins = pairs_df.iloc[:, 1].values  # Second column: Protein IDs

# Initialize KNN model
knn = KNeighborsRegressor(n_neighbors=10)

#### Implement and run cross-validation

In [98]:
# Implement and run the requested cross-validation. Report and interpret its results.

##### Leave-One-Drug-Out CV

In [83]:
# LODO-CV Implementation
logo_drug = LeaveOneGroupOut()
predictions_drug = np.zeros_like(y, dtype=float)

for train_idx, test_idx in logo_drug.split(X, y, groups=drugs):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train = y[train_idx]

    knn.fit(X_train, y_train)
    predictions_drug[test_idx] = knn.predict(X_test)

c_index_drug = compute_c_index(y, predictions_drug)
print(f"Leave-One-Drug-Out C-index: {c_index_drug:.3f}")

Leave-One-Drug-Out C-index: 0.520


<font color = "green">The cross-validation method used in the study was Leave-One-Out (LOO), in which a single drug-protein pair was left out at each iteration. This approach might make the model seem more accurate than it actually is in terms of its performance because similar drugs and proteins often appear both in training and test data. This leads to data leakage, causing the model to perform well in cross-validation but fail on truly unseen drug-protein interactions. To fix this issue, I used Leave-One-Drug-Out cross-validation, which systematically leaves out all interactions of a single drug during training. This simulates a real-world scenario, where we need to predict the association of completely unseen drugs based on previously measured interactions. <br>
If we talk about the c-index value which is 0.520, it means the model is only slightly better than random at predicting association for new drugs, showing poor generalization. The high accuracy in LOO-CV mentioned in the report, was likely due to data leakage, as similar drugs were both in training and testing. However, in case of LODO-CV, it tested the model on completely new drugs, revealing that it was memorizing rather than learning patterns. This makes the model unreliable for selecting new drug candidates and could lead to wasted lab resources. To improve, better molecular representations and more advanced models such as deep learning should be considered.

##### Leave-One-Group-Out CV

In [70]:
# Combine drug and protein identifiers to form groups
groups = [f"{drug}_{protein}" for drug, protein in zip(pairs_df[0], pairs_df[1])]

# Leave-One-Group-Out CV (LOGO-CV)
logo_group = LeaveOneGroupOut()
predictions_group = np.zeros_like(y, dtype=float)

for train_idx, test_idx in logo_group.split(X, y, groups=groups):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train = y[train_idx]

    knn.fit(X_train, y_train)
    predictions_group[test_idx] = knn.predict(X_test)

# Compute C-index for LOGO-CV
c_index_group = compute_c_index(y, predictions_group)
print(f"Leave-One-Group-Out C-index: {c_index_group:.3f}")

Leave-One-Group-Out C-index: 0.830


<font color = "green">LOGO-CV was used to test the model’s ability to predict association for completely new drug-protein pairs, ensuring it generalizes beyond known drugs or proteins. Unlike LODO-CV, which only removes drugs, LOGO-CV removes both drugs and proteins in each fold, making it a stricter and more realistic evaluation for drug discovery. The C-index of 0.830 suggests that the model ranks affinities well even for unseen drug-protein pairs, indicating it has learned meaningful patterns rather than just memorizing data. This result is significantly better than the LODO-CV c-index of 0.520, which showed poor generalization to new drugs alone. The strong LOGO-CV performance suggests the model can be useful for prioritizing new drug-protein interactions, but further validation and improvements, such as refining molecular features, could still enhance reliability.

Conclusion: <br>
<font color = green> As per the above calulcation, LODO-CV shows poor generalization to new drugs with a c-index value of 0.520, meaning it would likely waste resources if used for predicting associations of completely unseen drugs. However, its performance improves significantly when tested on entirely new drug-protein pairs using LOGO-CV. The c-index value of 0.830 suggests that it has learned some meaningful patterns.